In [1]:
import pandas as pd
import duckdb
import os
from requests_html import HTMLSession
from openai import OpenAI
import json
import re

from dotenv import load_dotenv
load_dotenv()

True

In [112]:
session = HTMLSession()

r = session.get('https://cifrhs.salud.gob.mx/site1/enarm/reportes_academicos.html')

all_links = list(r.html.absolute_links)

In [113]:
list_of_links = ", ".join(all_links)

In [114]:
list_of_links

'https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E39_enarm_ra_2015.doc, https://cifrhs.salud.gob.mx/index.html, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E46_enarm_ra_2022.pdf, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E34_enarm_ra_2010.doc, https://cifrhs.salud.gob.mx/site1/enarm/2024/docs/2024-ENARx_Reporte-Academico.pdf, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E36_enarm_ra_2012.doc, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E43_enarm_ra_2019.doc, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E37_enarm_ra_2013.doc, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E32_enarm_ra_2008.doc, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E45_enarm_ra_2021.pdf, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E26_enarm_ra_2002.doc, https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E27_enarm_ra_2003.

In [ ]:
# Initialize the OpenAI client
OPENAI_API_KEY  = os.getenv('openai_personal_key')
client = OpenAI(api_key=OPENAI_API_KEY)

# Create the message structure with role and content
messages = [
    {"role": "system", "content": """
        You are a helpful assistant that returns python dictionaries with this structure:
    { "2023":{
        "reporte_academico":"https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E47_enarm_ra_2023.pdf",
    },
     "2022":{
        "reporte_academico":"https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E47_enarm_ra_2022.pdf",
    },
     "2021":{
        "reporte_academico":"https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E47_enarm_ra_2021.pdf",
    }
    }
     based on the user input, which will be a list of urls. Pick the best url for each key and ONLY RETRUN THE DICTIONARY.
     Look out from the years 2024 to 2012
     """},
    {"role": "user", "content": list_of_links}
]

# Create the completion request
completion = client.chat.completions.create(
    model='gpt-4o-mini',  # Using GPT-4 model
    messages=messages,    # Pass the structured messages
    max_tokens=600         # Limit to a short response
)

# Print the response
print(completion.choices[0].message.content)

In [2]:
data_per_year = {
    "2024":{
        "base_url":"https://cifrhs.salud.gob.mx/site1/enarm/2024/",
        "reporte_academico":"https://cifrhs.salud.gob.mx/site1/enarm/2024/docs/2024-ENARx_Reporte-Academico.pdf",
        "plazas_ofertadas":"https://cifrhs.salud.gob.mx/site1/enarm/2024/docs/2024-ENARM-Plazas-lugares.pdf",
        "plazas_medicos_seleccionados":"https://cifrhs.salud.gob.mx/site1/enarm/2024/docs/2024-ENARM-Plazas-mex.pdf",
        "puntajes_max_min":"https://cifrhs.salud.gob.mx/site1/enarm/2024/docs/2024-ENARM-puntajes.pdf"
    },
    "2023":{
        "base_url":"https://cifrhs.salud.gob.mx/site1/enarm/2023.html",
        "reporte_academico":"https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E47_enarm_ra_2023.pdf",
        "plazas_ofertadas":"https://cifrhs.salud.gob.mx/site1/enarm/2023/docs/2023-ENARM-Plazas_mex-lugares-ext.pdf",
        "plazas_medicos_seleccionados":"https://cifrhs.salud.gob.mx/site1/enarm/2023/docs/2023-ENARM-Plaza-Especialidad-Mex.pdf",
        "puntajes_max_min":"https://cifrhs.salud.gob.mx/site1/enarm/2023/docs/E47_puntajes_max_min_2023.pdf"
    },
    "2022":{
        "base_url":"https://cifrhs.salud.gob.mx/site1/enarm/2022.html",
        "reporte_academico":"https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E46_enarm_ra_2022.pdf",
        "plazas_ofertadas":"https://cifrhs.salud.gob.mx/site1/enarm/_2022/docs/E46_plazas_mex_lugares_ext_2022.pdf",
        "plazas_medicos_seleccionados":"https://cifrhs.salud.gob.mx/site1/enarm/_2022/docs/E46_plazas_ins_mex_2022.pdf",
        "puntajes_max_min":"https://cifrhs.salud.gob.mx/site1/enarm/_2022/docs/E46_puntajes_max_min_2022.pdf"
    },
    "2021": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2021.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E45_enarm_ra_2021.pdf",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2021/E45_plazas_mex_lugares_ext_2021.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2021/E45_folio_sel_mex_2021-2.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2021/E45_puntajes_max_min_2021-2.pdf"
    },
    "2020": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2020.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E44_enarm_ra_2020.docx",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2020/E44_plazas_mex_lugares_ext_2020.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2020/E44_folio_sel_mex_2020.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2020/E44_puntajes_max_min_2020.pdf"
    },
    "2019": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2019.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E43_enarm_ra_2019.doc",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2019/E43_plazas_mex_lugares_ext_2019.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2019/E43_folio_sel_mex_2019.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2019/E43_puntajes_max_min_2019.pdf"
    },
    "2018": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2018.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E42_enarm_ra_2018.doc",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2018/E42_plazas_mex_lugares_ext_2018.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2018/E42_plazas_ins_mex_2018.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2018/E42_puntajes_max_min_2018.pdf"
    },
    "2017": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2017.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E41_enarm_ra_2017.doc",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2017/E41_plazas_mex_lugares_ext_2017.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2017/E41_folio_sel_ext_2017.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2017/E41_puntajes_max_min_2017.pdf"
    },
    "2016": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2016.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E40_enarm_ra_2016.doc",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2016/E40_plazas_mex_lugares_ext_2016.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2016/E40_folio_sel_mex_2016.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2016/E40_puntajes_max_min_2016.pdf"
    },
    "2015": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2015.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E39_enarm_ra_2015.doc",
        "plazas_ofertadas": "",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2015/E39_folios_sel_mex_2015.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2015/E39_plazas_mex_lugaresd_ext_2015.pdf"
    },
    "2014": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2014.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E38_enarm_ra_2014.doc",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2014/E38_plazas_ofer_mex_ext_2014.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2014/E38_folios_sel_mex_2014.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2014/E38_max_min_mex_ext_sel_2014.pdf"
    },
    "2013": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2013.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E37_enarm_ra_2013.doc",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2013/E37_plazas_ofer_mex_ext_2013.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2013/E37_folios_sel_mex_2013.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2013/E37_max_min_mex_ext_sel.pdf"
    },
    "2012": {
        "base_url": "https://cifrhs.salud.gob.mx/site1/enarm/2012.html",
        "reporte_academico": "https://cifrhs.salud.gob.mx/site1/enarm/docs/reportes_academicos/E36_enarm_ra_2012.doc",
        "plazas_ofertadas": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2012/E36_plazas_ofer_mex_ext_2012.pdf",
        "plazas_medicos_seleccionados": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2012/E36_max_min_mex_ext_sel_2012.pdf",
        "puntajes_max_min": "https://cifrhs.salud.gob.mx/site1/enarm/docs/2012/E36_max_min_mex_ext_sel_2012.pdf"
    }
}

In [6]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

endpoint = "https://gmarrcognitiveservices.cognitiveservices.azure.com/"
key = "9yr7GIgC9LSE8cpGUo2BNezqYQpzmEXdI8qXS8hAhhw5pINhB7smJQQJ99BDACYeBjFXJ3w3AAALACOGpKb1"
document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Extracting table data from the document
def extract_table_data_from_url(url):
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", AnalyzeDocumentRequest(url_source=url)
    )
    result = poller.result()

    table_data = []
    j = 0
    for table_idx, table in enumerate(result.tables):
        j = j+1
        for cell in table.cells:
            j = j+1
            table_data.append({\
                "pol":j,
                "row_index": cell.row_index,
                "column_index": cell.column_index,
                "content": cell.content
            })
    return table_data



In [16]:
baseU = data_per_year["2022"]['plazas_medicos_seleccionados']
yoyo = extract_table_data_from_url("https://videostostore.blob.core.windows.net/enarmimages/2022_2.png")

In [17]:
print(baseU)
print(yoyo)

https://cifrhs.salud.gob.mx/site1/enarm/_2022/docs/E46_plazas_ins_mex_2022.pdf
[{'pol': 2, 'row_index': 0, 'column_index': 0, 'content': 'HOSPITAL ANGELES DE PUEBLA'}, {'pol': 3, 'row_index': 0, 'column_index': 1, 'content': ''}, {'pol': 4, 'row_index': 0, 'column_index': 2, 'content': '3'}, {'pol': 5, 'row_index': 0, 'column_index': 3, 'content': ''}, {'pol': 6, 'row_index': 0, 'column_index': 4, 'content': ''}, {'pol': 7, 'row_index': 0, 'column_index': 5, 'content': ''}, {'pol': 8, 'row_index': 0, 'column_index': 6, 'content': ''}, {'pol': 9, 'row_index': 0, 'column_index': 7, 'content': ''}, {'pol': 10, 'row_index': 0, 'column_index': 8, 'content': ''}, {'pol': 11, 'row_index': 0, 'column_index': 9, 'content': ''}, {'pol': 12, 'row_index': 0, 'column_index': 10, 'content': ''}, {'pol': 13, 'row_index': 0, 'column_index': 11, 'content': ''}, {'pol': 14, 'row_index': 0, 'column_index': 12, 'content': ''}, {'pol': 15, 'row_index': 0, 'column_index': 13, 'content': ''}, {'pol': 16, 'ro

In [18]:
# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(yoyo)

# Extract column names from the first 5 elements
columns = df['content'].to_list()[0:29]

# Extract the remaining elements
data = df['content'].to_list()[29:]

# Reshape the data into rows of 4 elements each
rows = [data[i:i+29] for i in range(0, len(data), 29)]

# Create the DataFrame
new_df = pd.DataFrame(rows, columns=columns)

In [19]:
specialties = ["Centro",
               "ANATOMÍA PATOLÓGICA", "ANESTESIOLOGÍA", "AUDIOLOGÍA, OTONEUROLOGÍA Y FONIATRÍA", 
               "CALIDAD DE LA ATENCIÓN CLÍNICA", 
               "CIRUGÍA GENERAL", "EPIDEMIOLOGÍA", "GENÉTICA MÉDICA", "GERIATRÍA", "GINECOLOGÍA Y OBSTETRICIA", 
               "IMAGENOLOGÍA DIAGNÓSTICA Y TERAPÉUTICA", "MEDICINA DE LA ACTIVIDAD FÍSICA Y DEPORTIVA", 
               "MEDICINA DE REHABILITACIÓN", "MEDICINA DE URGENCIAS", "MEDICINA DEL TRABAJO Y AMBIENTAL", 
               "MEDICINA FAMILIAR", "MEDICINA INTERNA", "MEDICINA LEGAL",
               "MEDICINA NUCLEAR E IMAGENOLOGÍA MOLECULAR", "MEDICINA PREVENTIVA", 
               "NEUMOLOGÍA", "OFTALMOLOGÍA", "OTORRINOLARINGOLOGÍA Y CIRUGÍA DE CABEZA Y CUELLO", 
               "PATOLOGÍA CLÍNICA", "PEDIATRÍA", "PSIQUIATRÍA", "RADIO ONCOLOGÍA", "TRAUMATOLOGÍA Y ORTOPEDIA","TOTAL" ]

# Make the current columns the first row
new_df.loc[-1] = new_df.columns  # Add a new row at the top with current column names
new_df.index = new_df.index + 1  # Shift index
new_df.sort_index(inplace=True)  # Reorder rows

# Assign the variable specialties as the new column names
new_df.columns = specialties
new_df.index = new_df.index + 1  # Shift index
new_df.sort_index(inplace=True)  # Reorder rows

new_df = new_df.replace(r'(\d+)\n:.*', r'\1', regex=True)
new_df = new_df.replace(r'(\d+)\s*:unselected:', r'\1', regex=True)
new_df = new_df.replace(r'^:unselected:$', '0', regex=True)

In [406]:
# # Example map of special characters to digits
# char_to_digit = {
#     'Ʒ': '3',
#     # Add others if needed
# }

# # Apply the special mapping before the existing regex replacements
# def replace_special_chars(val):
#     # Match a single char followed by \n:selected:
#     match = re.match(r'^([^\w\s])\n:selected:', str(val))
#     if match:
#         char = match.group(1)
#         return char_to_digit.get(char, val)  # fallback to original if not mapped
#     return val

# # Apply the function to the entire dataframe
# new_df = new_df.map(replace_special_chars)

# new_df = new_df.replace(r'(\d+)\n:.*', r'\1', regex=True)
# new_df = new_df.replace(r':unselected:', '0', regex=True)
# new_df = new_df.replace(r'(\d+)\n:selected:', r'\1', regex=True)


In [20]:
new_df

,Centro,ANATOMÍA PATOLÓGICA,ANESTESIOLOGÍA,"AUDIOLOGÍA, OTONEUROLOGÍA Y FONIATRÍA",CALIDAD DE LA ATENCIÓN CLÍNICA,CIRUGÍA GENERAL,EPIDEMIOLOGÍA,GENÉTICA MÉDICA,GERIATRÍA,GINECOLOGÍA Y OBSTETRICIA,...,MEDICINA PREVENTIVA,NEUMOLOGÍA,OFTALMOLOGÍA,OTORRINOLARINGOLOGÍA Y CIRUGÍA DE CABEZA Y CUELLO,PATOLOGÍA CLÍNICA,PEDIATRÍA,PSIQUIATRÍA,RADIO ONCOLOGÍA,TRAUMATOLOGÍA Y ORTOPEDIA,TOTAL
1,HOSPITAL ANGELES DE PUEBLA,,3,,,,,,,,...,,,,,,,,,,3
2,HOSPITAL ANGELES DEL CARMEN,,3,,,,,,,,...,,,,,,,,,,6
3,HOSPITAL ANGELES ACOXPA,,4,,,,0,0,,,...,,,,,,,,,,4
4,HOSPITAL ANGELES DEL PEDREGAL,,3,,,6,0,0,,4,...,,,0,,,3,,,,29
5,HOSPITAL ANGELES LEÓN,,,,,,,,,,...,,,,,,,,,,4
6,HOSPITAL ANGELES LOMAS,,4,,,3,,,,5,...,,,,,,,,,,20
7,HOSPITAL ANGELES METROPOLITANO,,3,,,3,,,,,...,,,,,,,,,,11
8,HOSPITAL ANGELES MÉXICO,,,,,,,,,5,...,,,,,,,,,,5
9,HOSPITAL ÁNGELES MOCEL,,3,,,4,,,0,0,...,0,,,,0,,,,3,18
10,HOSPITAL ESPAÑOL,,4,,,10,,,3,6,...,0,,,3,,12,3,,4,65


In [15]:
new_df1 = new_df.copy()

In [21]:
new_df2 = new_df.copy()

In [423]:
new_df3 = new_df.copy()

In [373]:
new_df4 = new_df.copy()

In [22]:
att = pd.concat([new_df1,new_df2])

In [23]:
att.reset_index(drop=True, inplace=True)

In [24]:
att.to_csv('/Users/gerardomartinez/Desktop/Projects/enarm/additional_docs/centros_salud/2022.csv', index=False)